In [ ]:
from llm_utils.activation_generator import ActivationGenerator
from factorization.hierchichal_snmf import train_hierarchical_nmf
from factorization.seminmf import NMFSemiNMF
from data_utils.concept_dataset import SupervisedConceptDataset
import random
import numpy as np
import torch

# Tutorial Notebook on training and analyzing recursive SNMF

I think most things are self-explanatory (with added comments where needed). Only thing that needs explanation is the NMFSemiNMF class which holds the SNMF implementation.

- After calling fit, model.F.T holds the factors or you can call model.H (which just transposes for you)

- It trains using multiplicative updates (read paper SNMF paper for details)
- Fit is called for you in this notebook, for training SNMF on its own see another notebook

### Device
If Macbook then chooses MPS, otherwise tries GPU

In [2]:
if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

Device: mps


### Configuration
- layers can be a list to produce activations from multiple layers, I suggest for this notebook to just choose a single layer
- data device is meant to reduce GPU usage, offloads unused data to CPU (if CPU is chosen)
- model device is what device to actually factorize with, use the fastest hardware you have (GPU/MPS)
- factorization mode is what activations to extract 
    - mlp: MLP activations post non-linearity
    - mlp_out: activations at the output of the MLP
    - residual: the activations of the residual stream at the end of the layer pass

In [ ]:
# path to data
data_path = "data/hier_concepts.json"

# name of model, must be supported by transformer lens
model_name = "gpt2-large"

# layers of model to inspect
layers = [8]

# Device to load data to, default is CPU and only factorization and generation occurs on GPU 
data_device = 'cpu'

# Device to load model to for generation
model_device = device

# factorization mode, factorize residual or mlp layers
factorization_mode = 'mlp'


### Set Seed for reproducibility

In [4]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # For multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False  # Ensures deterministic behavior

set_seed(42)

### Generate Activations and Load Data

In [5]:
act_generator = ActivationGenerator(model_name, model_device=model_device, data_device=data_device, mode=factorization_mode)

Loaded pretrained model gpt2-large into HookedTransformer


#### Dataset Abstraction
- Supports a couple dataset formats, this specific one requires labels (prompt, label)
- Same file also has ConceptDataset which all the objects know how to utilize and only needs prompts

In [6]:
dataset = SupervisedConceptDataset(data_path)

In [ ]:
len(dataset)

1079

Produce activations, also outputs the frequencies of each token for weighting, can ignore

In [ ]:
activations, freq = act_generator.generate_multiple_layer_activations_and_freq(dataset, layers)

Building vocab frequency:   0%|          | 0/216 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Generating multi-layer activations with freq: 100%|██████████| 216/216 [00:22<00:00,  9.41it/s]


#### Hierarchichal SNMF Config
- ranks: The sizes of the SNMFs we are stacking
- nmf_device: Use your GPU
- fitting_device: Use your GPU
- max_iterations_per_layer: How many multiplicative update iterations per SNMF
- epoch_to_epoch_tol: tolerance on loss change for early stopping, I reccomend setting to a negative number (SNMF trains fast)
- lr: learning rate, not very sensitive to it. Just choose something around 1e-2 to 1e-4

In [ ]:
# model parameters
ranks = [400, 200, 100, 50]
nmf_device = 'mps' # reccomended to only use GPU for inference and fitting
fitting_device = device
max_iterations_per_layer = 3000
epoch_to_epoch_tol = 1e-6
lr = 1e-3

### Hierarchichal Training Function
Trains the Recursive SNMF based on the configs

- ft_iters is how many fine tuning iterations to do at the end (based on the paper), doesn't matter too much. I didn't explore this parameter too much.
- cls keep the NMFSemiNMF, only one supported (as opposed to NMF)
- sparsity is how much sparsity to apply to the SNMF (based on your use, for MLP activations I reccomened 0.01 for large models)

Returns
- h_nmf the hierarchichal SNMF abstraction 
- nmf_list: A list that contains the actual SNMF objects (This is what we use in analysis after)

In [22]:
(h_nmf, nmf_list) = train_hierarchical_nmf(activations[0], ranks, device='mps', pretrain_kwargs={
    'max_iter': max_iterations_per_layer, 'tol': epoch_to_epoch_tol,
}, ft_iters=2500, cls=NMFSemiNMF, cls_args={'sparsity': 0.01}, fine_tune=True)

Iter    0: loss=1231878.875000  (best=1231878.875000, no_improve=0)
Iter    2: loss=1240159.125000  (best=1230895.500000, no_improve=1)
Iter   50: loss=1167112.125000  (best=1167112.125000, no_improve=0)
Iter   78: loss=1141245.375000  (best=1140723.625000, no_improve=1)
Iter  100: loss=1132285.125000  (best=1132285.125000, no_improve=0)
Iter  138: loss=1114310.250000  (best=1114305.000000, no_improve=1)
Iter  145: loss=1113525.000000  (best=1113292.000000, no_improve=1)
Iter  147: loss=1113207.125000  (best=1113188.125000, no_improve=1)
Iter  150: loss=1112918.625000  (best=1112918.625000, no_improve=0)
Iter  158: loss=1111259.375000  (best=1111177.250000, no_improve=1)
Iter  173: loss=1106958.250000  (best=1106928.000000, no_improve=1)
Iter  180: loss=1105444.125000  (best=1105440.750000, no_improve=1)
Iter  187: loss=1103925.875000  (best=1103924.750000, no_improve=1)
Iter  200: loss=1101755.125000  (best=1101755.125000, no_improve=0)
Iter  201: loss=1101917.750000  (best=1101755.12

#### Save if you want, I use Pickle, I'm sure you can adapt it to work with state

In [10]:
import pickle
with open('hier_nmf.pkl', 'rb') as f:
    # pickle.dump((h_nmf, nmf_list), f)
    (h_nmf, nmf_list) = pickle.load(f)

## Analysis

#### Automatically Identifying Hierarchies
- We identify hierarchies by inspecting the coefficient matrix.
- We threshold so that we draw an edge between subsequent SNMF features if the feature is used by the previous feature (previous SNMF) for reconstruction. We get these values from the coefficient matrices.
- The following code does this in an automated way

In [11]:
import torch
import numpy as np

def get_top_activating_indices_hierarchical(
    layers: list,
    concept_idx: int,
    layer_idx: int,
    num_samples: int = 10,
    minimal_activation: float = 0.0
):
    """
    For a hierarchical NMF defined by `layers` (where
      layers[0].W has shape (n_samples,   r1),
      layers[1].W has shape (r1,          r2),
      …,
      layers[L].W has shape (r_L,         hidden_dim)
    ), compute the “effective activation” of each sample for
    the concept `concept_idx` at layer `layer_idx` by
    multiplying
    
       M = W₀ @ W₁ @ … @ W_{layer_idx}
    
    then taking column `concept_idx` of M.

    Parameters
    ----------
    layers : list of NMFMU
      Pretrained layers 0…L, each with `.W` as a torch.Tensor.
    concept_idx : int
      Which concept/column to inspect at the specified layer.
    layer_idx : int
      Index of the layer (0-based).
    num_samples : int
      How many top‐activating samples to return.
    minimal_activation : float
      Threshold: ignore any sample with activation ≤ this.

    Returns
    -------
    top_indices : list[int]
      The sample indices (0…n_samples-1) of the top activations.
    activations : list[float]
      The corresponding activation values.
    """
    # 1) Multiply W0 @ W1 @ … @ W_layer_idx
    M = layers[0].W  # (n_samples, r1)
    for i in range(1, layer_idx + 1):
        M = M @ layers[i].W

    # 2) Extract the desired concept column
    #    M has shape (n_samples, r_{layer_idx+1})
    col = M[:, concept_idx]              # (n_samples,)
    vals = col.detach().cpu().numpy()    # numpy array for sorting

    # 3) Get indices of the top activations
    sorted_idxs = np.argsort(vals)[::-1]  # descending order
    top_indices = []
    activations = []

    for idx in sorted_idxs:
        if vals[idx] <= minimal_activation:
            break
        top_indices.append(int(idx))
        activations.append(float(vals[idx]))
        if len(top_indices) >= num_samples:
            break

    return top_indices, activations


def print_logit_diff(model, logits_before, logits_after):
    # Select the logits for the last token in the sequence (shape: [vocab_size])
    logits_before_last = logits_before[0, -1, :]
    logits_after_last = logits_after[0, -1, :]

    # Compute the difference in logits (after - before)
    delta_logits = (logits_after_last - logits_before_last)

    # Get the top 10 tokens with the largest positive increase
    top_increases, top_indices = torch.topk(abs(delta_logits), k=20)

    print("Tokens with the highest logit increase after intervention:")
    for token_id, increase in zip(top_indices, top_increases):
        # Convert token ID to string using your model's tokenizer
        # Here we assume feature_processor._model.to_str_tokens returns a readable token string
        token_str = model.to_str_tokens([token_id])
        print(f"Token: {token_str}, Increase: {increase.item():.4f}")

In [12]:
import torch
import numpy as np

def build_concept_tree(
    layers: list,
    concept_idx: int,
    layer_idx: int,
    top_k_factors: int = 3,
    top_k_tokens: int = 10,
    minimal_activation: float = 0.0
):
    # 1) Get the raw top activations (no filtering)
    indices, acts = get_top_activating_indices_hierarchical(
        layers, concept_idx, layer_idx,
        num_samples=top_k_tokens * 5,  # grab extra so filtering has room
        minimal_activation=0.0
    )

    # 2) Compute a per-node threshold
    if 0.0 < minimal_activation < 1.0:
        node_max = max(acts) if acts else 0.0
        token_thresh = minimal_activation * node_max
    else:
        token_thresh = minimal_activation

    # 3) Filter token activations and cap at top_k_tokens
    filtered_tokens = [(i, a) for i, a in zip(indices, acts) if a >= token_thresh]
    filtered_tokens.sort(key=lambda x: x[1], reverse=True)
    filtered_tokens = filtered_tokens[:top_k_tokens]

    node = {
        'layer': layer_idx,
        'concept': concept_idx,
        'top_indices': filtered_tokens,
        'children': []
    }

    # 4) Recurse for factor-level children if not at bottom
    if layer_idx > 0:
        W = layers[layer_idx].W.detach().cpu().numpy()
        contrib = W[:, concept_idx]  # each lower-layer factor’s importance

        # compute child threshold
        if 0.0 < minimal_activation < 1.0:
            child_thresh = minimal_activation * contrib.max()
        else:
            child_thresh = minimal_activation

        # collect all valid factors, then cap at top_k_factors
        sorted_factors = np.argsort(contrib)[::-1]
        valid_factors = [f for f in sorted_factors if contrib[f] >= child_thresh]
        top_factors = valid_factors[:top_k_factors]

        # stop this branch if there's only one child to create
        if len(top_factors) <= 1:
            return node

        for f_idx in top_factors:
            child = build_concept_tree(
                layers,
                concept_idx=int(f_idx),
                layer_idx=layer_idx-1,
                top_k_factors=top_k_factors,
                top_k_tokens=top_k_tokens,
                minimal_activation=minimal_activation
            )
            node['children'].append(child)

    return node


#### Visualizing
- We use networkx to visualize the trees generated by the previous function
- We label the nodes using the collaped coefficient matrices (connects deeper features to the activations)
- For earlier layers this is much easier to do as the features are more syntactic and there is less mixing so the labels are a lot more obvious.

In [13]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib as mpl
import textwrap

# ———————————————————————————————————————————————
# Global styling for publication–quality figures (bigger text!)
# ———————————————————————————————————————————————
mpl.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Times New Roman", "Palatino", "serif"],
    "font.size": 60,                # base font size
    "axes.titlesize": 35,           # axes title
    "axes.labelsize": 35,           # x/y labels
    "xtick.labelsize": 35,          # tick labels
    "ytick.labelsize": 35,
    "legend.fontsize": 35,
    "figure.titlesize": 35,
    "text.usetex": False,
    "figure.dpi": 300,
    "savefig.format": "pdf",
})

def plot_concept_tree(node,
                      vocab,
                      figsize=(20, 25),
                      vert_gap=1.2,
                      wrap_width=30,
                      fontsize=18,
                      save_path=None):
    """
    Plots a hierarchical concept tree using networkx + matplotlib
    with publication–ready aesthetics and large, readable text.

    Args:
        node (dict):      The root node (from build_concept_tree).
        vocab (List[str]):Mapping from token-indices to token strings.
        figsize (tuple):  Figure size in inches.
        vert_gap (float): Vertical gap between levels.
        wrap_width (int): Max chars per line in node labels.
        fontsize (int):   Font size for node text boxes.
        save_path (str):  If provided, save SVG/PDF to this path.
    """
    # 1) Build DiGraph
    G = nx.DiGraph()
    def _add(n):
        nid = id(n)
        G.add_node(nid, data=n)
        for c in n.get('children', []):
            G.add_edge(nid, id(c))
            _add(c)
    _add(node)

    # 2) Compute layout (Graphviz 'dot' or fallback)
    try:
        pos = nx.nx_agraph.graphviz_layout(G, prog="dot")
    except Exception:
        def _hierarchy_pos(root):
            pos = {}
            x_counter = [0]
            def recurse(n, depth):
                nid = id(n)
                kids = n.get('children', [])
                if not kids:
                    pos[nid] = (x_counter[0], -depth * vert_gap)
                    x_counter[0] += 1
                else:
                    start = x_counter[0]
                    for c in kids:
                        recurse(c, depth + 1)
                    end = x_counter[0] - 1
                    pos[nid] = ((start + end) / 2, -depth * vert_gap)
            recurse(node, 0)
            return pos
        pos = _hierarchy_pos(node)

    # 3) Prepare labels (wrapped)
    labels = {}
    for nid, data in G.nodes(data=True):
        top = data['data'].get('top_indices', [])
        tokens = [vocab[idx] for idx, _ in top]
        txt = ",".join(tokens) if tokens else "(no tokens)"
        labels[nid] = textwrap.fill(txt, width=wrap_width)

    # 4) Draw graph
    plt.figure(figsize=figsize)
    ax = plt.gca()
    nx.draw(
        G, pos,
        with_labels=False,
        arrows=True,
        arrowstyle="-|>",
        arrowsize=10,
        width=1.5,
        edge_color="gray",
        ax=ax
    )

    # Node labels with big text
    for nid, (x, y) in pos.items():
        # … inside your for-loop over nid, (x,y) in pos.items():
        ax.text(
            x, y, labels[nid],
            ha="center", va="center",
            fontsize=fontsize,        # e.g. pass 24 or 30 when calling
            linespacing=1.5,           # 1.0 is default; >1 spreads lines out
            bbox=dict(
                boxstyle="round,pad=1.0",  # bump from 0.6 → bigger box
                fc="white",
                ec="black",
                lw=1.5
            )
        )


    # Clean up
    ax.margins(0.1)
    ax.set_axis_off()
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, bbox_inches="tight")
    plt.show()


In [14]:
from llm_utils.activation_generator import extract_token_ids_sample_ids_and_labels

tokens, sample_ids, labels = extract_token_ids_sample_ids_and_labels(dataset, act_generator)

Extracting token IDs: 100%|██████████| 216/216 [00:00<00:00, 895.61it/s]


In [15]:
def generate_token_contexts(tokens, sample_ids, act_generator):
    # Define how many tokens before and after to include in the context
    context_window = 15

    token_ds = []
    for i in range(len(tokens)):
        current_sample_id = sample_ids[i]
        # Convert the current token to its string representation
        token_str = act_generator.model.to_str_tokens([tokens[i]])[0][0]
        
        # Determine the start and end indices for the context window
        start = max(0, i - context_window)
        end = min(len(tokens), i + context_window + 1)
        
        # Get the string representation for each token in the context
        context_tokens = [
            act_generator.model.to_str_tokens([tokens[j]])[0][0] for j in range(start, end) if sample_ids[j] == current_sample_id
        ]
        
        # Join the context tokens into a single string
        context_str = "".join(context_tokens)
        
        # Append the (token, context) tuple to the list
        token_ds.append((token_str, context_str))

    return token_ds

token_ds = generate_token_contexts(tokens, sample_ids, act_generator)

In [16]:
def print_concept_tree(node, token_ds, indent: int = 0):
    """
    Recursively print a concept tree node.

    node: dict returned by build_concept_tree(...)
    token_ds: list-like where token_ds[i] == [token_string, some_meta]
    indent: current indentation level
    """
    prefix = "    " * indent
    # Header for this factor
    print(f"{prefix}Layer {node['layer']}, Concept {node['concept']}:")
    # Its top activating tokens
    for sample_idx, act in node['top_indices']:
        token, meta = token_ds[sample_idx]
        print(f"{prefix}    Token: {token}\t|| Context: {meta} || Act: {act}")
    print()  # blank line before children

    # Recurse into children
    for child in node.get('children', []):
        print_concept_tree(child, token_ds, indent + 1)


Print all the concept trees in a layer where the layer_idx defines the root of the tree 

In [ ]:
# layers: your list of trained NMF layers (0…L)
# root_idx: the highest‐level concept you want to explore
# L = len(layers)-1
for i in range(200):
    tree = build_concept_tree(
        layers=nmf_list,
        concept_idx=i,
        layer_idx=2,
        top_k_factors=4,   # how many sub‐factors per node
        top_k_tokens=10,    # how many sample indices per node
        minimal_activation=0.15
    )
    print_concept_tree(tree, token_ds)


Layer 2, Concept 0:
    Token:  window	|| Context: She tapped nervously on the windowpane, waiting for her ride to arrive. || Act: 0.17544816434383392
    Token:  window	|| Context: She tapped impatiently on the windowpane, waiting for him to arrive. || Act: 0.17486488819122314
    Token:  windows	|| Context: Dust motes danced in the light filtering through the grimy windows. || Act: 0.17089727520942688
    Token:  window	|| Context: Rain streaked down the windowpane, blurring the outside world. || Act: 0.16908162832260132
    Token:  windows	|| Context: She nervously tapped her fingers on the windowsill, peering out the windows. || Act: 0.16815140843391418
    Token:  window	|| Context: A gentle breeze drifted through the open window, carrying the scent of rain. || Act: 0.16607162356376648
    Token:  window	|| Context: He peered through the window, hoping to catch a glimpse of her. || Act: 0.16500379145145416
    Token:  window	|| Context: The sunlight streamed through the window pan

IndexError: index 100 is out of bounds for dimension 1 with size 100

This code actually outputs an image of a selected tree

In [125]:
plot_concept_tree(tree, [x for x,y in token_ds],figsize=(50, 60), vert_gap=3, wrap_width=10, save_path="/Users/orshafran/Desktop/college/thesis/results_final/discussion/rand.pdf", fontsize=20)